In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### import train and test data sets

In [16]:
data_train = pd.read_csv("train.csv")
data_train.head(2)

Comment  Outcome     Id
0  use variables in the outer function instead of...        1  18966
1  remember your knuth. "premature optimization i...        1  12559

In [17]:
print(data_train.isna().sum().sum())
print(data_train.shape)

0
(50678, 3)


In [18]:
data_test = pd.read_csv("test.csv")
data_test.head(2)

Comment     Id
0  i use the tail() function: tail(vector, n=1) t...  69098
1  if x is your data.frame (or matrix) then x[ ,a...  72599

In [19]:
print(data_test.isna().sum().sum())
print(data_test.shape)

0
(36431, 2)


In [20]:
data_train['text'] = data_train['Comment'].str.lower()

In [21]:
data_train.head(2)

Comment  Outcome     Id  \
0  use variables in the outer function instead of...        1  18966   
1  remember your knuth. "premature optimization i...        1  12559   

                                                text  
0  use variables in the outer function instead of...  
1  remember your knuth. "premature optimization i...

In [22]:
data_test['text'] = data_test['Comment'].str.lower()

In [23]:
data_test.head(2)

Comment     Id  \
0  i use the tail() function: tail(vector, n=1) t...  69098   
1  if x is your data.frame (or matrix) then x[ ,a...  72599   

                                                text  
0  i use the tail() function: tail(vector, n=1) t...  
1  if x is your data.frame (or matrix) then x[ ,a...

### build neural network via word2vec

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import gensim

from collections import Counter
import pickle
import itertools

Using TensorFlow backend.


In [26]:
X_train = data_train.text
y_train = data_train.Outcome

X_test = data_test.text

In [27]:
X_train[0]

'use variables in the outer function instead of global variables. this gets you the best of both approaches: you\'re not mutating global state, and you\'re not copying a big wad of data. if you have to exit early, just return the partial results. (see the "scope" section in the r manual: http://cran.r-project.org/doc/manuals/r-intro.html#scope)'

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

145807


In [29]:
sequence_length = 100

X_train2 = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen=sequence_length)

In [30]:
X_train2.shape

(50678, 100)

In [31]:
encoder = LabelEncoder()
encoder.fit(y_train.tolist())

y_train2 = encoder.transform(y_train.tolist())
y_train2 = y_train2.reshape(-1, 1)

In [32]:
w2v_size = 20

In [33]:
lstm_nn = Sequential()
lstm_nn.add(Embedding(vocab_size, w2v_size,
                    input_length=sequence_length, trainable=True))
lstm_nn.add(Dropout(0.5))
lstm_nn.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
lstm_nn.add(Dense(1, activation="sigmoid"))

lstm_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 20)           2916140   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                14200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 2,930,391
Trainable params: 2,930,391
Non-trainable params: 0
_________________________________________________________________


In [34]:
lstm_nn.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [35]:
history = lstm_nn.fit(X_train2, y_train2, batch_size=2048, epochs=8,
                    validation_split=0.1)

/Users/rui/pyenv/bt5153/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 45610 samples, validate on 5068 samples
Epoch 1/8
45610/45610 [==============================] - 37s 817us/step - loss: 0.6904 - accuracy: 0.5292 - val_loss: 0.7510 - val_accuracy: 0.0067
Epoch 2/8
45610/45610 [==============================] - 33s 716us/step - loss: 0.6707 - accuracy: 0.6035 - val_loss: 0.7290 - val_accuracy: 0.4272
Epoch 3/8
45610/45610 [==============================] - 32s 707us/step - loss: 0.6282 - accuracy: 0.6651 - val_loss: 0.6885 - val_accuracy: 0.5631
Epoch 4/8
45610/45610 [==============================] - 32s 709us/step - loss: 0.5868 - accuracy: 0.7008 - val_loss: 0.7268 - val_accuracy: 0.5450
Epoch 5/8
45610/45610 [==============================] - 33s 717us/step - loss: 0.5536 - accuracy: 0.7285 - val_loss: 0.6590 - val_accuracy: 0.6172
Epoch 6/8
45610/45610 [==============================] - 34s 742us/step - loss: 0.5192 - accuracy: 0.7541 - val_loss: 0.6851 - val_accuracy: 0.6036
Epoch 7/8
45610/45610 [==============================] - 44s 96

In [36]:
def predict(text, seq_lenth):    
    # padding the X test (text) to have the same sequence length
    XX_test = pad_sequences(tokenizer.texts_to_sequences(text),
                            maxlen=seq_lenth)
    score = lstm_nn.predict(XX_test)
    label = [1 if prob>0.5 else 0 for prob in score ]
    return label

In [37]:
y_pred = predict(X_test, sequence_length)

In [38]:
submission_lstm = pd.DataFrame({
    "Id": data_test.Id,
    "Outcome": y_pred
})
submission_lstm.to_csv("submission_rnn.csv", index=False)

In [39]:
submission_lstm.Outcome.value_counts()

1    19414
0    17017
Name: Outcome, dtype: int64